<a href="https://colab.research.google.com/github/joaobat/Classificao-R-git/blob/master/04_DSA_Cap12_BOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='blue'>Data Science Academy - Machine Learning</font>

# <font color='blue'>Capítulo 12 - Processamento de Linguagem Natural</font>

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.6.9


Obs: Este é um material de bônus incluído neste curso. PyTorch é estudado em detalhes no curso <a href="https://www.datascienceacademy.com.br/course?courseid=deep-learning-frameworks">Deep Learning Frameworks</a> e aplicado em PLN no curso <a href="https://www.datascienceacademy.com.br/course?courseid=processamento-de-linguagem-natural-e-reconhecimento-de-voz">Processamento de Linguagem Natural</a>.

## Modelo de Classificação de Idiomas de Sentenças com Bag of Words e PyTorch

![title](imagens/bow.png)

In [4]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install torch==1.5.0

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [ ]:
# Instala o PyTorch
!pip install -q -U torch torchvision

In [2]:
# Imports
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torch import nn, optim

In [5]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

pandas 1.1.4
torch  1.7.0+cu101
numpy  1.18.5
Data Science Academy


### Preparando os Dados

In [6]:
# Dados de treino
dados_treino = [("Tenho vinte paginas de leitura".lower().split(), "Portuguese"),
                ("I will visit the library".lower().split(), "English"),
                ("I am reading a book".lower().split(), "English"),
                ("This is my favourite chapter".lower().split(), "English"),
                ("Estou na biblioteca lendo meu livro preferido".lower().split(), "Portuguese"),
                ("Gosto de livros sobre viagens".lower().split(), "Portuguese")]

In [7]:
dados_treino

[(['tenho', 'vinte', 'paginas', 'de', 'leitura'], 'Portuguese'),
 (['i', 'will', 'visit', 'the', 'library'], 'English'),
 (['i', 'am', 'reading', 'a', 'book'], 'English'),
 (['this', 'is', 'my', 'favourite', 'chapter'], 'English'),
 (['estou', 'na', 'biblioteca', 'lendo', 'meu', 'livro', 'preferido'],
  'Portuguese'),
 (['gosto', 'de', 'livros', 'sobre', 'viagens'], 'Portuguese')]

In [8]:
# Dados de teste
dados_teste = [("Estou lendo".lower().split(), "Portuguese"),
               ("This is not my favourite book".lower().split(), "English")]

In [9]:
dados_teste

[(['estou', 'lendo'], 'Portuguese'),
 (['this', 'is', 'not', 'my', 'favourite', 'book'], 'English')]

In [10]:
# Prepara o dicionário do vocabulário

# Dicionário para o vocabulário
dict_vocab = {}

# Contadoor
i = 0

# Loop pelos dados de treino e teste
for palavras, idiomas in dados_treino + dados_teste:
    for palavra in palavras:
        if palavra not in dict_vocab:
            dict_vocab[palavra] = i
            i += 1

In [11]:
# Visualiza o vocabulário
print(dict_vocab)

{'tenho': 0, 'vinte': 1, 'paginas': 2, 'de': 3, 'leitura': 4, 'i': 5, 'will': 6, 'visit': 7, 'the': 8, 'library': 9, 'am': 10, 'reading': 11, 'a': 12, 'book': 13, 'this': 14, 'is': 15, 'my': 16, 'favourite': 17, 'chapter': 18, 'estou': 19, 'na': 20, 'biblioteca': 21, 'lendo': 22, 'meu': 23, 'livro': 24, 'preferido': 25, 'gosto': 26, 'livros': 27, 'sobre': 28, 'viagens': 29, 'not': 30}


In [12]:
# Tamanho do corpus
tamanho_corpus = len(dict_vocab)

In [13]:
tamanho_corpus

31

In [14]:
# Número de idiomas
idiomas = 2

In [15]:
# Índice para os idiomas
label_index = {"Portuguese": 0, "English": 1}

### Construção do Modelo

In [16]:
# Classe para o modelo BOW de classificação
class ModeloBOW(nn.Module):  

    # Método construtor
    def __init__(self, lista_idiomas, tamanho_do_corpus):
        super(ModeloBOW, self).__init__()
        self.linear = nn.Linear(tamanho_do_corpus, lista_idiomas)

    # Feed Forward
    def forward(self, bow_vec):
        return F.log_softmax(self.linear(bow_vec), dim = 1)

In [18]:
# Função para criar o vetor BOW necessário para o treinamento
def cria_bow_vetor(sentence, word_index):
    word_vec = torch.zeros(tamanho_corpus)
    for word in sentence:
        word_vec[dict_vocab[word]] += 1
    return word_vec.view(1, -1)

In [19]:
# Função para criar a variável target
def cria_target(label, label_index):
    return torch.LongTensor([label_index[label]])

In [20]:
# Cria o modelo
modelo = ModeloBOW(idiomas, tamanho_corpus)

In [21]:
# Função de perda (loss)
loss_function = nn.NLLLoss()

In [22]:
# Otimizador
optimizer = optim.SGD(modelo.parameters(), lr = 0.1)

### Treinamento do Modelo

In [23]:
# Loop de treinamentoo
for epoch in range(100):
    
    for sentence, label in dados_treino:

        modelo.zero_grad()

        bow_vec = cria_bow_vetor(sentence, dict_vocab)
        target = cria_target(label, label_index)

        log_probs = modelo(bow_vec)

        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()
        
    if epoch % 10 == 0:
        print('Epoch: ', str(epoch+1),', Loss: ' + str(loss.item()))

Epoch:  1 , Loss: 0.47799405455589294
Epoch:  11 , Loss: 0.07790250331163406
Epoch:  21 , Loss: 0.04109955206513405
Epoch:  31 , Loss: 0.027839526534080505
Epoch:  41 , Loss: 0.021033329889178276
Epoch:  51 , Loss: 0.01689634472131729
Epoch:  61 , Loss: 0.014117375016212463
Epoch:  71 , Loss: 0.012122409418225288
Epoch:  81 , Loss: 0.010620973072946072
Epoch:  91 , Loss: 0.009450225159525871


### Previsões e Avaliação do Modelo

In [24]:
# Função para previsões
def faz_previsao(data):

    with torch.no_grad():
        sentence = data[0]
        label = data[1]
        bow_vec = cria_bow_vetor(sentence, dict_vocab)
        log_probs = modelo(bow_vec)
        print(sentence)
        print('Probabilidade de ser o label: ' + label, 'é igual a: ',  np.exp(log_probs))

In [25]:
# Previsão com a primeira sentença de teste
faz_previsao(dados_teste[0])

['estou', 'lendo']
Probabilidade de ser o label: Portuguese é igual a:  tensor([[0.8052, 0.1948]])


In [26]:
dados_teste[0]

(['estou', 'lendo'], 'Portuguese')

In [27]:
# Previsão com a segunda sentença de teste
faz_previsao(dados_teste[1])

['this', 'is', 'not', 'my', 'favourite', 'book']
Probabilidade de ser o label: English é igual a:  tensor([[0.0069, 0.9931]])


In [28]:
dados_teste[1]

(['this', 'is', 'not', 'my', 'favourite', 'book'], 'English')

### Previsões com Novas Frases

In [29]:
# Nova frase
novas_frases = [("Tenho livros sobre viagens".lower().split(), "Portuguese"),
                ("Estou escrevendo".lower().split(), "Portuguese"),
                ("Gosto de biblioteca".lower().split(), "Portuguese")]

In [30]:
novas_frases

[(['tenho', 'livros', 'sobre', 'viagens'], 'Portuguese'),
 (['estou', 'escrevendo'], 'Portuguese'),
 (['gosto', 'de', 'biblioteca'], 'Portuguese')]

In [31]:
faz_previsao(novas_frases[0])

['tenho', 'livros', 'sobre', 'viagens']
Probabilidade de ser o label: Portuguese é igual a:  tensor([[0.9519, 0.0481]])


In [ ]:
faz_previsao(novas_frases[1])

KeyError: 'escrevendo'

In [32]:
faz_previsao(novas_frases[2])

['gosto', 'de', 'biblioteca']
Probabilidade de ser o label: Portuguese é igual a:  tensor([[0.9545, 0.0455]])


# Fim